In [ ]:
%load_ext autoreload
%autoreload 2

from typing import Union
import os
import shutil
import sys

import numpy as np
import pandas as pd

sys.path.append("../../../")

from examples.logistic_regression.kdd99.dataset import kdd99_csv_to_dataframe, kdd99_df_to_dataset, ATTACK_CLASSIFICATIONS
from examples.logistic_regression.model import LogisticRegressionTrainingModel, LogisticRegressionTrainingModelParams, LogisticRegressionEvaluationModelParams

In [ ]:
class Fake:  # wrapper to denote to generate data that is representative but not necessarily correct
    def __init__(self, fraction, *, attack_type: str):
        self.fraction = fraction
        self.attack_type = attack_type

def generate_rel_split(num_special_agents: int, num_normal_agents: int, special_agent_factor: float):
    denom = num_special_agents * special_agent_factor + num_normal_agents
    answer = []
    for _ in range(num_special_agents):
        answer.append(special_agent_factor/denom)
    for _ in range(num_normal_agents):
        answer.append(1/denom)
    return answer

# dict of names to split sequences.
# each subsequence is a sequence of tuples representing what % of the (false data, true data) each client should have. True data and false data %s must each sum to 1.0
SPLITS = {
    # symmetric and equal splits
    "sym_eq_1": (1.0,),  # one client with everything
    "sym_eq_25": [1/25] * 25,  # 25 clients with equal splits
    "sym_eq_50": [1/50] * 50,  # 50 clients with equal splits
    "sym_eq_100": [1/100] * 100,  # 50 clients with equal splits
    
    "sym_neq_50x0.125": generate_rel_split(10, 40, .125),
    "sym_neq_50x0.25": generate_rel_split(10, 40, .25),
    "sym_neq_50x0.5": generate_rel_split(10, 40, .5),
    "sym_neq_50x0.75": generate_rel_split(10, 40, .75),
    "sym_neq_50x1.5": generate_rel_split(10, 40, 1.5),
    "sym_neq_50x2": generate_rel_split(10, 40, 2),
    "sym_neq_50x4": generate_rel_split(10, 40, 4),

    # attack splits
    "attack_random_50_one":       [1/50] * 49 + [Fake(1/50, attack_type='random')],
    "attack_random_50_quarter":   [1/50] * 38 + [Fake(1/50, attack_type='random')] * 12,
    "attack_random_50_half":      [1/50] * 26 + [Fake(1/50, attack_type='random')] * 24,
    "attack_inverted_50_one":     [1/50] * 49 + [Fake(1/50, attack_type='inverted')],
    "attack_inverted_50_quarter": [1/50] * 38 + [Fake(1/50, attack_type='inverted')] * 12,
    "attack_inverted_50_half":    [1/50] * 26 + [Fake(1/50, attack_type='inverted')] * 24,
}
VALIDATION_FRACTIONS = [0.2]  # put aside 20% of the records for validation. Not gonna sweep on this


DATA_10PERCENT = "kddcup.data_10_percent"
DATA_FULL = "kddcup.data"
DATASET_NAME = DATA_FULL  # Should be DATA_10PERCENT or DATA_FULL
CUSTOM_TEST = True  # True to split DATASET_NAME into 2/3 train/validate 1/3 test; False to use all of DATASET_NAME for the clients and DATA_CORRECTED for test
DATA_FOLDER = f"/path/to/data/root/folder/{DATASET_NAME}/"
DATASET_PATH = os.path.join(DATA_FOLDER, DATASET_NAME)
DATA_CORRECTED = os.path.join(DATA_FOLDER, "corrected")
os.makedirs(DATA_FOLDER, exist_ok=True)

In [ ]:
if not os.path.exists(DATASET_PATH):
    zipped_file = DATASET_PATH + ".gz"
    if not os.path.exists(zipped_file):
        !wget -O "{zipped_file}" http://kdd.ics.uci.edu/databases/kddcup99/{DATASET_NAME}.gz
    !gunzip "{zipped_file}"
if not os.path.exists(DATA_CORRECTED):
    zipped_file = os.path.join(DATA_FOLDER, "corrected.gz")
    if not os.path.exists(zipped_file):
        !wget -O "{zipped_file}" http://kdd.ics.uci.edu/databases/kddcup99/corrected.gz
    !gunzip "{zipped_file}"

In [ ]:
df_normal = kdd99_csv_to_dataframe(DATASET_PATH)
df_corrected = kdd99_csv_to_dataframe(DATA_CORRECTED)
kdd99_df_to_dataset(df_normal).save(os.path.join(DATA_FOLDER, "normal.dat"))
kdd99_df_to_dataset(df_corrected).save(os.path.join(DATA_FOLDER, "corrected_all.dat"))
df_normal

In [ ]:
df_normal.info()

In [ ]:
# Remove all new attacks from the corrected dataset
known_corrected_df_slices = []
for attack_name, attack_type in ATTACK_CLASSIFICATIONS:
    known_corrected_df_slices.append(df_corrected[df_corrected["attack_type"] == attack_name])
known_corrected_df = pd.concat(known_corrected_df_slices)
kdd99_df_to_dataset(known_corrected_df).save(os.path.join(DATA_FOLDER, "corrected_known.dat"))
known_corrected_df

In [ ]:
df_normal_shuffle = df_normal.sample(frac=1)
df_normal_shuffle

In [ ]:
df_tv = df_normal_shuffle.iloc[:int(2/3*len(df_normal_shuffle))]
df_test = df_normal_shuffle.iloc[int(2/3*len(df_normal_shuffle)):]
kdd99_df_to_dataset(df_tv).save(os.path.join(DATA_FOLDER, "normal_tv.dat"))
kdd99_df_to_dataset(df_test).save(os.path.join(DATA_FOLDER, "normal_test.dat"))

In [ ]:
def slice_df(df: pd.DataFrame, start_i: int, fraction: Union[float, Fake]):
    if isinstance(fraction, float):
        num_records = int(fraction * len(df))
        new_df = df.iloc[start_i:start_i+num_records]
        new_start_i = start_i + num_records
        return (new_start_i, new_df)
    else:
        assert isinstance(fraction, Fake)
        num_records = int(fraction.fraction * len(df))
        real_df = df.iloc[start_i:start_i+num_records]
        if fraction.attack_type == "random":
            fake_data = real_df.to_numpy(copy=True)
            for i, column in enumerate(real_df.columns):
                size = len(real_df)
                try:
                    mean = real_df[column].mean()
                    std = real_df[column].std()
                    fake_data[:, i] = np.random.normal(mean, std, size=size)
                except TypeError:  # it's discrete
                    names = []
                    counts = []
                    for name, count in real_df[column].value_counts().items():
                        assert isinstance(name, str)
                        names.append(name)
                        counts.append(count)
                    np_counts = np.array(counts, dtype=np.float)
                    np_counts /= np.sum(np_counts)
                    fake_data[:, i] = np.random.choice(names, size=size, replace=True, p=np_counts)
                    
            fake_df = pd.DataFrame(data=fake_data, columns=real_df.columns).infer_objects()
        else:
            assert fraction.attack_type == "inverted"
            fake_df = real_df.copy(deep=True)
            fake_df.loc[fake_df['attack_type'] != 'normal.', 'attack_type'] = 'attack.'
            fake_df.loc[fake_df['attack_type'] == 'normal.', 'attack_type'] = 'FAKE'
            fake_df.loc[fake_df['attack_type'] == 'attack.', 'attack_type'] = 'normal.' 
            fake_df.loc[fake_df['attack_type'] == 'FAKE', 'attack_type'] = 'attack.'
        new_start_i = start_i + num_records
        return (new_start_i, fake_df)

In [ ]:
if CUSTOM_TEST:
    df_shuffle = df_tv
    print("Using 66% of noraml data")
else:
    df_shuffle = df_normal_shuffle
    print("Using 100% of normal data")
print("Length of dataset", len(df_shuffle))
for split_name, split in SPLITS.items():
    i = 0
    for validation_size in VALIDATION_FRACTIONS:
        split_folder = os.path.join(DATA_FOLDER, f"split_{split_name}_validation_fraction_{validation_size}")
        shutil.rmtree(split_folder, ignore_errors=True)
    for client_i, fraction in enumerate(split):
        assert i < len(df_shuffle)
        i, new_df = slice_df(df_tv, i, fraction)
        for validation_size in VALIDATION_FRACTIONS:
            train_size = int((1 - validation_size) * len(new_df))
            train_df = new_df[:train_size]
            validation_df = new_df[train_size:]
            split_folder = os.path.join(DATA_FOLDER, f"split_{split_name}_validation_fraction_{validation_size}")
            client_folder = os.path.join(split_folder, f"client_{client_i}")
            os.makedirs(client_folder)
            print("Saving to ", client_folder)
            kdd99_df_to_dataset(train_df).save(os.path.join(client_folder, "train.dat"))
            kdd99_df_to_dataset(validation_df).save(os.path.join(client_folder, "validation.dat"))
            kdd99_df_to_dataset(new_df).save(os.path.join(client_folder, "score.dat"))